## Adapter

A construct which adapts an existing interface X to conform
to the required interface Y.

### no-caching

In [1]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    
def draw_point(p):
    print('.', end='')
    
    
class Line:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
class Rectangle(list):
    def __init__(self, x, y, width, height):
        super().__init__()
        self.append(Line(Point(x, y), Point(x + width, y)))
        self.append(Line(Point(x + width, y), Point(x + width, y + height)))
        self.append(Line(Point(x, y), Point(x, y + height)))
        self.append(Line(Point(x, y + height), Point(x + width, y + height)))
              

class LineToPointAdapter(list):
    count = 0
    
    def __init__(self, line):
        super().__init__()
        self.count += 1
        
        print(f'{self.count}: Generating points for line '
              f'[{line.start.x},{line.start.y}]→'
              f'[{line.end.x},{line.end.y}]')

        left = min(line.start.x, line.end.x)
        right = max(line.start.x, line.end.x)
        top = min(line.start.y, line.end.y)
        bottom = min(line.start.y, line.end.y)

        if right - left == 0:
            for y in range(top, bottom):
                self.append(Point(left, y))
        elif line.end.y - line.start.y == 0:
            for x in range(left, right):
                self.append(Point(x, top))            
            
def draw(rcs):
    print('\n\n--- Drawing some stuff ---\n')
    for rc in rcs:
        for line in rc:
            adapter = LineToPointAdapter(line)
            for p in adapter:
                draw_point(p)
    
        
rs = [
    Rectangle(1, 1, 10, 10),
    Rectangle(3, 3, 6, 6)
]
draw(rs)            



--- Drawing some stuff ---

1: Generating points for line [1,1]→[11,1]
..........1: Generating points for line [11,1]→[11,11]
1: Generating points for line [1,1]→[1,11]
1: Generating points for line [1,11]→[11,11]
..........1: Generating points for line [3,3]→[9,3]
......1: Generating points for line [9,3]→[9,9]
1: Generating points for line [3,3]→[3,9]
1: Generating points for line [3,9]→[9,9]
......

### with-caching

In [11]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    
def draw_point(p):
    print('.', end='')
    
    
class Line:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
class Rectangle(list):
    def __init__(self, x, y, width, height):
        super().__init__()
        self.append(Line(Point(x, y), Point(x + width, y)))
        self.append(Line(Point(x + width, y), Point(x + width, y + height)))
        self.append(Line(Point(x, y), Point(x, y + height)))
        self.append(Line(Point(x, y + height), Point(x + width, y + height)))
              

class LineToPointAdapter:
    cache = {}
    
    def __init__(self, line):
        self.h = hash(line)
        if self.h in self.cache:
            return
        
        print(f'Generating points for line '
              f'[{line.start.x},{line.start.y}]→'
              f'[{line.end.x},{line.end.y}]')

        left = min(line.start.x, line.end.x)
        right = max(line.start.x, line.end.x)
        top = min(line.start.y, line.end.y)
        bottom = min(line.start.y, line.end.y)
        
        points = []
        if right - left == 0:
            for y in range(top, bottom):
                points.append(Point(left, y))
        elif line.end.y - line.start.y == 0:
            for x in range(left, right):
                points.append(Point(x, top))            
        
        self.cache[self.h] = points
        
    def __iter__(self):
        return iter(self.cache[self.h])
            
        
            
def draw(rcs):
    print('\n\n--- Drawing some stuff ---\n')
    for rc in rcs:
        for line in rc:
            adapter = LineToPointAdapter(line)
            for p in adapter:
                draw_point(p)
    
        
rs = [
    Rectangle(1, 1, 10, 10),
    Rectangle(3, 3, 6, 6)
]
draw(rs) 
draw(rs)



--- Drawing some stuff ---

Generating points for line [1,1]→[11,1]
..........Generating points for line [11,1]→[11,11]
Generating points for line [1,1]→[1,11]
Generating points for line [1,11]→[11,11]
..........Generating points for line [3,3]→[9,3]
......Generating points for line [9,3]→[9,9]
Generating points for line [3,3]→[3,9]
Generating points for line [3,9]→[9,9]
......

--- Drawing some stuff ---

................................